In [1]:
# Imports 
import pandas as pd
import numpy as np
import yfinance as yf


In [2]:
# Fetch data function, change symbol/datetime here
def fetch_forex_data():
    
    data = yf.download(tickers = 'EURUSD=X' ,period ='1mo', interval = '5m') 
    df = pd.DataFrame(data)
    return df

In [3]:

# Function to calculate Simple Moving Average (SMA)
def calculate_sma(df, window):
    return df['Close'].rolling(window=window).mean()

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(df, window):
    delta = df['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Moving Average Convergence Divergence (MACD)
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Function to generate trading signals
def generate_signals(df):
    df['SMA'] = calculate_sma(df, window=20)
    df['RSI'] = calculate_rsi(df, window=14)
    df['MACD'], df['MACD_SIGNAL'] = calculate_macd(df)
    
    df['Signal'] = 0
    df['Signal'] = np.where((df['Close'] > df['SMA']) & (df['RSI'] < 70) & (df['MACD'] > df['MACD_SIGNAL']), 1, df['Signal'])
    df['Signal'] = np.where((df['Close'] < df['SMA']) & (df['RSI'] > 30) & (df['MACD'] < df['MACD_SIGNAL']), -1, df['Signal'])
    df['Position'] = df['Signal'].diff()
    return df
def test_signals(df):
    df['Return'] = df['Close'].pct_change()
    df['Strategy_Return'] = df['Return'] * df['Signal'].shift(1)
    
    df['Cumulative_Market_Return'] = (1 + df['Return']).cumprod()
    df['Cumulative_Strategy_Return'] = (1 + df['Strategy_Return']).cumprod()
    
    total_market_return = df['Cumulative_Market_Return'].iloc[-1]
    total_strategy_return = df['Cumulative_Strategy_Return'].iloc[-1]
    
    winning_trades = len(df[(df['Signal'] == 1) & (df['Return'] > 0)]) + len(df[(df['Signal'] == -1) & (df['Return'] < 0)])
    losing_trades = len(df[(df['Signal'] == 1) & (df['Return'] < 0)]) + len(df[(df['Signal'] == -1) & (df['Return'] > 0)])
    total_trades = winning_trades + losing_trades
    
    accuracy = winning_trades / total_trades if total_trades > 0 else 0
    
    return {
        'Total Market Return': total_market_return,
        'Total Strategy Return': total_strategy_return,
        'Winning Trades': winning_trades,
        'Losing Trades': losing_trades,
        'Total Trades': total_trades,
        'Accuracy': accuracy
    }

In [4]:
df = fetch_forex_data()
df = generate_signals(df)
print(test_signals(df))
pd.DataFrame.to_csv(df, "annotated.csv")

[*********************100%%**********************]  1 of 1 completed

{'Total Market Return': 0.9862705400958585, 'Total Strategy Return': 1.003093346992058, 'Winning Trades': 1338, 'Losing Trades': 810, 'Total Trades': 2148, 'Accuracy': 0.6229050279329609}
